In [1]:
# Set Variables
## Fixed
tpm_hist = {}  # record tp metric values for trials
tp_metric = 'avgwl'  # specified trade_param_metric: ratio avg value win/loss
## Settable by User
n_trials = 10  # number of HP optimization runs
total_timesteps = 2000 # per HP optimization run
total_episodes = 10
## Logging callback params
lc_threshold=1e-5
lc_patience=15
lc_trial_number=5

In [2]:
# !pip install pyfolio-reloaded  #original pyfolio no longer maintained
# !pip install optuna
# !pip install -U "ray[rllib]"
# !pip install plotly
# !pip install ipywidgets
# !pip install -U kaleido   # enables saving plots to file

In [3]:
! pip install -U kaleido

In [4]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import optuna
from pathlib import Path
from ddpg_torch import Agent
import gym
import numpy as np
import pandas as pd
from utils import plotLearning
# from config import *

import config_tickers
from download_data import process_data
from stock_trading_env import StockTradingEnv
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import data_split
from trade_stocks import trade_on_test_df
# from finrl.agents.stablebaselines3.models import DRLAgent

import os
import ray
import kaleido
import itertools

import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f'Torch device: {device}')
from plot import get_comparison_df, backtest_stats, backtest_plot, get_daily_return, get_baseline

Torch device: cuda


/home/neeraj/anaconda3/envs/ddpg_trading/lib/python3.8/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [5]:
import random

In [6]:
# data
# data_path = './data/data_processed_DOW_30_TICKER_2009-01-01_to_2022-07-31.csv'
# processed_full = pd.read_csv(data_path, index_col='Unnamed: 0')

In [7]:
# processed_full.shape

In [8]:
TRAIN_START_DATE = '2009-01-01'

In [9]:
TRAIN_END_DATE = '2016-01-01'

In [10]:
TRADE_START_DATE = '2016-01-01'
TRADE_END_DATE = '2020-05-10'

In [11]:
ticker_name_from_config_tickers = 'DOW_30_TICKER'

TRAIN_CSV_NAME = f'./data/train_{ticker_name_from_config_tickers}_{TRAIN_START_DATE}_to_{TRAIN_END_DATE}.csv'
TRADE_CSV_NAME = f'./data/trade_{ticker_name_from_config_tickers}_{TRADE_START_DATE}_to_{TRADE_END_DATE}.csv'

In [12]:
INDICATORS = [
            'macd',
            'boll_ub',
            'boll_lb',
            'rsi_30',
            'cci_30',
            'dx_30',
            'close_30_sma',
            'close_60_sma'
]


In [13]:
# train = data_split(processed_full, '2009-01-01','2020-07-01')
# trade = data_split(processed_full, '2020-05-01','2021-10-01')

In [14]:
# train = data_split(processed_full, TRAIN_START_DATE, TRAIN_END_DATE)
# trade = data_split(processed_full, TRADE_START_DATE, TRADE_END_DATE)
# print(f'Number of training samples: {len(train)}')
# print(f'Number of testing samples: {len(trade)}')

In [15]:
print('reading csvs...')
train = pd.read_csv(TRAIN_CSV_NAME, index_col='Unnamed: 0')
trade = pd.read_csv(TRADE_CSV_NAME, index_col='Unnamed: 0')

print(f'Train shape: {train.shape} \n Trade shape: {trade.shape}')

reading csvs...
Train shape: (51098, 18) 
 Trade shape: (31726, 18)


In [16]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 291


In [17]:
#Defining the environment kwargs

# env_kwargs = {
#     "hmax": 100, 
#     "initial_amount": 1000000, 
#     "buy_cost_pct": 0.001,
#     "sell_cost_pct": 0.001,
#     "state_space": state_space, 
#     "stock_dim": stock_dimension, 
#     "tech_indicator_list": config.INDICATORS, 
#     "action_space": stock_dimension, 
#     "reward_scaling": 1e-4
    
# }
# #Instantiate the training gym compatible environment
# e_train_gym = StockTradingEnv(df = train, **env_kwargs)
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension


env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

env_train, _ = e_train_gym.get_sb_env()

#Instantiate the trading environment
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = None, **env_kwargs)

In [18]:
SEED = 0

In [19]:
np.random.seed(SEED)
e_train_gym.seed(SEED)

torch.manual_seed(SEED)
random.seed(SEED)

In [20]:
# agent = DRLAgent(env = env_train)

## Trade performance code

In [17]:
#Main method
# Calculates Trade Performance for Objective
# Called from objective method
# Returns selected trade perf metric(s)
# Requires actions and associated prices

def calc_trade_perf_metric(df_actions, 
                           df_prices_trade,
                           tp_metric,
                           dbg=False):
  
    df_actions_p, df_prices_p, tics = prep_data(df_actions.copy(),
                                                df_prices_trade.copy())
    # actions predicted by trained model on trade data
    df_actions_p.to_csv('df_actions.csv') 

    
    # Confirms that actions, prices and tics are consistent
    df_actions_s, df_prices_s, tics_prtfl = \
        sync_tickers(df_actions_p.copy(),df_prices_p.copy(),tics)
    
    # copy to ensure that tics from portfolio remains unchanged
    tics = tics_prtfl.copy()
    
    # Analysis is performed on each portfolio ticker
    perf_data= collect_performance_data(df_actions_s, df_prices_s, tics)
    # profit/loss for each ticker
    pnl_all = calc_pnl_all(perf_data, tics)
    # values for trade performance metrics
    perf_results = calc_trade_perf(pnl_all)
    df = pd.DataFrame.from_dict(perf_results, orient='index')
    
    # calculate and return trade metric value as objective
    m = calc_trade_metric(df,tp_metric)
    print(f'Ratio Avg Win/Avg Loss: {m}')
    k = str(len(tpm_hist)+1)
    # save metric value
    tpm_hist[k] = m
    return m

In [18]:
# Supporting methods
def calc_trade_metric(df,metric='avgwl'):
    '''# trades', '# wins', '# losses', 'wins total value', 'wins avg value',
       'losses total value', 'losses avg value'''
    # For this tutorial, the only metric available is the ratio of 
    #  average values of winning to losing trades. Others are in development.
    
    # some test cases produce no losing trades.
    # The code below assigns a value as a multiple of the highest value during
    # previous hp optimization runs. If the first run experiences no losses,
    # a fixed value is assigned for the ratio
    tpm_mult = 1.0
    avgwl_no_losses = 25
    if metric == 'avgwl':
        if sum(df['# losses']) == 0:
            try:
                return max(tpm_hist.values())*tpm_mult
            except ValueError:
                return avgwl_no_losses
        avg_w = sum(df['wins total value'])/sum(df['# wins'])
        avg_l = sum(df['losses total value'])/sum(df['# losses'])
        m = abs(avg_w/avg_l)

    return m


def prep_data(df_actions,
              df_prices_trade):
    
    df=df_prices_trade[['date','close','tic']]
    df['Date'] = pd.to_datetime(df['date'])
    df = df.set_index('Date')
    # set indices on both df to datetime
    idx = pd.to_datetime(df_actions.index, infer_datetime_format=True)
    df_actions.index=idx
    tics = np.unique(df.tic)
    n_tics = len(tics)
    print(f'Number of tickers: {n_tics}')
    print(f'Tickers: {tics}')
    dategr = df.groupby('tic')
    p_d={t:dategr.get_group(t).loc[:,'close'] for t in tics}
    df_prices = pd.DataFrame.from_dict(p_d)
    df_prices.index = df_prices.index.normalize()
    return df_actions, df_prices, tics


# prepares for integrating action and price files
def link_prices_actions(df_a,
                        df_p):
    cols_a = [t + '_a' for t in df_a.columns]
    df_a.columns = cols_a
    cols_p = [t + '_p' for t in df_p.columns]
    df_p.columns = cols_p
    return df_a, df_p


def sync_tickers(df_actions,df_tickers_p,tickers):
    # Some DOW30 components may not be included in portfolio
    # passed tickers includes all DOW30 components
    # actions and ticker files may have different length indices
    if len(df_actions) != len(df_tickers_p):
        msng_dates = set(df_actions.index)^set(df_tickers_p.index)
        try:
            #assumption is prices has one additional timestamp (row)
            df_tickers_p.drop(msng_dates,inplace=True)
        except:
            df_actions.drop(msng_dates,inplace=True)
    df_actions, df_tickers_p = link_prices_actions(df_actions,df_tickers_p)
    # identify any DOW components not in portfolio
    t_not_in_a = [t for t in tickers if t + '_a' not in list(df_actions.columns)]
  
    # remove t_not_in_a from df_tickers_p
    drop_cols = [t + '_p' for t in t_not_in_a]
    df_tickers_p.drop(columns=drop_cols,inplace=True)
    
    # Tickers in portfolio
    tickers_prtfl = [c.split('_')[0] for c in df_actions.columns]
    return df_actions,df_tickers_p, tickers_prtfl

def collect_performance_data(dfa,dfp,tics, dbg=False):
    
    perf_data = {}
    # In current version, files columns include secondary identifier
    for t in tics:
        # actions: purchase/sale of DOW equities
        acts = dfa['_'.join([t,'a'])].values
        # ticker prices
        prices = dfp['_'.join([t,'p'])].values
        # market value of purchases/sales
        tvals_init = np.multiply(acts,prices)
        d={'actions':acts, 'prices':prices,'init_values':tvals_init}
        perf_data[t]=d

    return perf_data


def calc_pnl_all(perf_dict, tics_all):
    # calculate profit/loss for each ticker
    print(f'Calculating profit/loss for each ticker')
    pnl_all = {}
    for tic in tics_all:
        pnl_t = []
        tic_data = perf_dict[tic]
        init_values = tic_data['init_values']
        acts = tic_data['actions']
        prices = tic_data['prices']
        cs = np.cumsum(acts)
        args_s = [i + 1 for i in range(len(cs) - 1) if cs[i + 1] < cs[i]]
        # tic actions with no sales
        if not args_s:
            pnl = complete_calc_buyonly(acts, prices, init_values)
            pnl_all[tic] = pnl
            continue
        # copy acts: acts_rev will be revised based on closing/reducing init positions
        pnl_all = execute_position_sales(tic,acts,prices,args_s,pnl_all)

    return pnl_all


def complete_calc_buyonly(actions, prices, init_values):
    # calculate final pnl for each ticker assuming no sales
    fnl_price = prices[-1]
    final_values = np.multiply(fnl_price, actions)
    pnl = np.subtract(final_values, init_values)
    return pnl


def execute_position_sales(tic,acts,prices,args_s,pnl_all):
  # calculate final pnl for each ticker with sales
    pnl_t = []
    acts_rev = acts.copy()
    # location of sales transactions
    for s in args_s:  # s is scaler
        # price_s = [prices[s]]
        act_s = [acts_rev[s]]
        args_b = [i for i in range(s) if acts_rev[i] > 0]
        prcs_init_trades = prices[args_b]
        acts_init_trades = acts_rev[args_b]
  
        # update actions for sales
        # reduce/eliminate init values through trades
        # always start with earliest purchase that has not been closed through sale
        # selectors for purchase and sales trades
        # find earliest remaining purchase
        arg_sel = min(args_b)
        # sel_s = len(acts_trades) - 1

        # closing part/all of earliest init trade not yet closed
        # sales actions are negative
        # in this test case, abs_val of init and sales share counts are same
        # zero-out sales actions
        # market value of sale
        # max number of shares to be closed: may be less than # originally purchased
        acts_shares = min(abs(act_s.pop()), acts_rev[arg_sel])

        # mv of shares when purchased
        mv_p = abs(acts_shares * prices[arg_sel])
        # mv of sold shares
        mv_s = abs(acts_shares * prices[s])

        # calc pnl
        pnl = mv_s - mv_p
        # reduce init share count
        # close all/part of init purchase
        acts_rev[arg_sel] -= acts_shares
        acts_rev[s] += acts_shares
        # calculate pnl for trade
        # value of associated purchase
        
        # find earliest non-zero positive act in acts_revs
        pnl_t.append(pnl)
    
    pnl_op = calc_pnl_for_open_positions(acts_rev, prices)
    #pnl_op is list
    # add pnl_op results (if any) to pnl_t (both lists)
    pnl_t.extend(pnl_op)
    #print(f'Total pnl for {tic}: {np.sum(pnl_t)}')
    pnl_all[tic] = np.array(pnl_t)
    return pnl_all


def calc_pnl_for_open_positions(acts,prices):
    # identify any positive share values after accounting for sales
    pnl = []
    fp = prices[-1] # last price
    open_pos_arg = np.argwhere(acts>0)
    if len(open_pos_arg)==0:
        return pnl # no open positions

    mkt_vals_open = np.multiply(acts[open_pos_arg], prices[open_pos_arg])
    # mkt val at end of testing period
    # treat as trades for purposes of calculating pnl at end of testing period
    mkt_vals_final = np.multiply(fp, acts[open_pos_arg])
    pnl_a = np.subtract(mkt_vals_final, mkt_vals_open)
    #convert to list
    pnl = [i[0] for i in pnl_a.tolist()]
    #print(f'Market value of open positions at end of testing {pnl}')
    return pnl


def calc_trade_perf(pnl_d):
    # calculate trade performance metrics
    perf_results = {}
    for t,pnl in pnl_d.items():
        wins = pnl[pnl>0]  # total val
        losses = pnl[pnl<0]
        n_wins = len(wins)
        n_losses = len(losses)
        n_trades = n_wins + n_losses
        wins_val = np.sum(wins)
        losses_val = np.sum(losses)
        wins_avg = 0 if n_wins==0 else np.mean(wins)
        #print(f'{t} n_wins: {n_wins} n_losses: {n_losses}')
        losses_avg = 0 if n_losses==0 else np.mean(losses)
        d = {'# trades':n_trades,'# wins':n_wins,'# losses':n_losses,
             'wins total value':wins_val, 'wins avg value':wins_avg,
             'losses total value':losses_val, 'losses avg value':losses_avg,}
        perf_results[t] = d
    return perf_results

## Tuning using Optuna}

In [19]:
# def sample_ddpg_params_all(trial:optuna.Trial,
#                            # fixed values from previous study
#                            learning_rate=0.0103,
#                            batch_size=128,
#                            buffer_size=int(1e6)):

#     gamma = trial.suggest_categorical("gamma", [0.94, 0.96, 0.98])
#     # Polyak coeff
#     tau = trial.suggest_categorical("tau", [0.08, 0.1, 0.12])

#     train_freq = trial.suggest_categorical("train_freq", [512,768,1024])
#     gradient_steps = train_freq
    
#     noise_type = trial.suggest_categorical("noise_type", ["ornstein-uhlenbeck", "normal", None])
#     noise_std = trial.suggest_categorical("noise_std", [.1,.2,.3] )

#     # NOTE: Add "verybig" to net_arch when tuning HER (see TD3)
#     net_arch = trial.suggest_categorical("net_arch", ["small", "big"])
#     # activation_fn = trial.suggest_categorical('activation_fn', [nn.Tanh, nn.ReLU, nn.ELU, nn.LeakyReLU])

#     net_arch = {
#         "small": [64, 64],
#         "medium": [256, 256],
#         "big": [512, 512],
#     }[net_arch]
  
#     hyperparams = {
#         "batch_size": batch_size,
#         "buffer_size": buffer_size,
#         "gamma": gamma,
#         "gradient_steps": gradient_steps,
#         "learning_rate": learning_rate,
#         "tau": tau,
#         "train_freq": train_freq,
#         #"noise_std": noise_std,
#         #"noise_type": noise_type,
        
#         "policy_kwargs": dict(net_arch=net_arch)
#     }
#     return hyperparams

In [19]:
def sample_ddpg_params(trial:optuna.Trial):
    # Size of the replay buffer
    buffer_size = trial.suggest_categorical("buffer_size", [int(1e4), int(1e5), int(1e6)])
#     tau = trial.suggest_categorical("tau", [0.01, 0.001, 0.1])
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-1)
#     learning_rate_critic = trial.suggest_loguniform("learning_rate_critic", 1e-5, 1e-1)
#     learning_rate_actor = 10**trial.suggest_int('logval', -5, 0)
#     learning_rate_critic = 10**trial.suggest_int('logval', -5, 0)
    
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256, 512])

    net_arch = trial.suggest_categorical("net_arch", ["small", "medium", "big"])

    net_arch = {
        "small": [64, 64],
        "medium": [256, 256],
        "default": [400, 400],
        "big": [512, 512],
    }[net_arch]

    return {
#             
#             "tau": tau,
            "learning_rate": learning_rate,
            "buffer_size": buffer_size,
            "batch_size": batch_size,
            "layer_1_size": net_arch[0],
            "layer_2_size": net_arch[1]}

In [20]:
class LoggingCallback:
    def __init__(self, threshold, trial_number, patience):
        '''
        threshold:int tolerance for increase in objective
        trial_number: int Prune after minimum number of trials
        patience: int patience for the threshold
        '''
        self.threshold = threshold
        self.trial_number  = trial_number
        self.patience = patience
        print(f'Callback threshold {self.threshold}, \
            trial_number {self.trial_number}, \
            patience {self.patience}')
        self.cb_list = [] #Trials list for which threshold is reached
        
    def __call__(self, study:optuna.study, frozen_trial:optuna.Trial):
        #Setting the best value in the current trial
        study.set_user_attr("previous_best_value", study.best_value)

        #Checking if the minimum number of trials have pass
        if frozen_trial.number >self.trial_number:
            previous_best_value = study.user_attrs.get("previous_best_value",None)
            #Checking if the previous and current objective values have the same sign
            if previous_best_value * study.best_value >=0:
                #Checking for the threshold condition
                if abs(previous_best_value-study.best_value) < self.threshold: 
                    self.cb_list.append(frozen_trial.number)
                    #If threshold is achieved for the patience amount of time
                    if len(self.cb_list)>self.patience:
                        print('The study stops now...')
                        print('With number',frozen_trial.number ,'and value ',frozen_trial.value)
                        print('The previous and current best values are {} and {} respectively'
                              .format(previous_best_value, study.best_value))
                        study.stop()

In [21]:
from IPython.display import clear_output
import sys   

os.makedirs("./models", exist_ok=True)

def objective(trial:optuna.Trial):
    #Trial will suggest a set of hyperparamters from the specified range

    # Optional to optimize larger set of parameters
    # hyperparameters = sample_ddpg_params_all(trial)

    # Optimize buffer size, batch size, learning rate
    hyperparameters = sample_ddpg_params(trial)
    print(f'Hyperparameters from objective: {hyperparameters}')
    
#     policy_kwargs = None  # default
#     if 'policy_kwargs' in hyperparameters.keys():
#         policy_kwargs = hyperparameters['policy_kwargs']
#         del hyperparameters['policy_kwargs']
    #print(f'Policy keyword arguments {policy_kwargs}')
#     model_ddpg = agent.get_model("ddpg",
#                                policy_kwargs = policy_kwargs,
#                                model_kwargs = hyperparameters )
#     #You can increase it for better comparison
#     trained_ddpg = agent.train_model(model=model_ddpg,
#                                    tb_log_name="ddpg",
#                                    total_timesteps=total_timesteps)
    ACTOR_LR = hyperparameters["learning_rate"]
    CRITIC_LR = hyperparameters["learning_rate"]
    BATCH_SIZE = hyperparameters["batch_size"]
    TAU = 0.001
    LAYER_1_SIZE = hyperparameters["layer_1_size"]
    LAYER_2_SIZE = hyperparameters["layer_2_size"]
    buffer_size = hyperparameters["buffer_size"]
    
    model_ddpg = Agent(alpha=ACTOR_LR, beta=CRITIC_LR, ckp_dir='./models', input_dims=state_space, tau=TAU,
              batch_size=BATCH_SIZE, layer1_size=LAYER_1_SIZE, layer2_size=LAYER_2_SIZE, max_size=buffer_size,
              n_actions=stock_dimension)
    
    trained_ddpg = model_ddpg.train_model(
                        total_episodes=total_episodes, train_from_scratch=True, 
                        env=e_train_gym, env_kwargs=env_kwargs, save_ckp=False, ckp_save_freq=0,
                        use_wandb=False)
    
#     trained_ddpg.save('models/ddpg_{}.pth'.format(trial.number))
    trained_ddpg.save_checkpoint(checkpoint_name='ddpg_{}.pt'.format(trial.number), last_episode=trained_ddpg.episode)
    
    clear_output(wait=True)
    #For the given hyperparamters, determine the account value in the trading period
    
#     df_account_value, df_actions = DRLAgent.DRL_prediction(
#     model=trained_ddpg, 
#     environment = e_trade_gym)
    df_account_value, df_actions, cumulative_rewards_test = trade_on_test_df(df=trade, 
                                                                             model=trained_ddpg, 
                                                                             train_df=train, 
                                                                             env_kwargs=env_kwargs)

    # Calculate trade performance metric
    # Currently ratio of average win and loss market values
    tpm = calc_trade_perf_metric(df_actions,trade,tp_metric)
    return tpm

#Create a study object and specify the direction as 'maximize'
#As you want to maximize sharpe
#Pruner stops not promising iterations
#Use a pruner, else you will get error related to divergence of model
#You can also use Multivariate samplere
#sampler = optuna.samplers.TPESampler(multivarite=True,seed=42)
sampler = optuna.samplers.TPESampler()
study = optuna.create_study(study_name="ddpg_study",direction='maximize',
                        sampler = sampler, pruner=optuna.pruners.HyperbandPruner())

logging_callback = LoggingCallback(threshold=lc_threshold,
                               patience=lc_patience,
                               trial_number=lc_trial_number)
#You can increase the n_trials for a better search space scanning
study.optimize(objective, n_trials=n_trials, catch=(ValueError,), callbacks=[logging_callback])

[I 2022-07-29 08:09:27,134] Trial 9 finished with value: 1.9684022492229998 and parameters: {'buffer_size': 1000000, 'learning_rate': 0.012392298037713417, 'batch_size': 128, 'net_arch': 'medium'}. Best is trial 3 with value: 2.91729969004257.


hit end!
Number of tickers: 29
Tickers: ['AAPL' 'AMGN' 'AXP' 'BA' 'CAT' 'CRM' 'CSCO' 'CVX' 'DIS' 'GS' 'HD' 'HON'
 'IBM' 'INTC' 'JNJ' 'JPM' 'KO' 'MCD' 'MMM' 'MRK' 'MSFT' 'NKE' 'PG' 'TRV'
 'UNH' 'V' 'VZ' 'WBA' 'WMT']
Calculating profit/loss for each ticker
Ratio Avg Win/Avg Loss: 1.9684022492229998


In [22]:
!ls models

ddpg_0.pt  ddpg_2.pt  ddpg_4.pt  ddpg_6.pt  ddpg_8.pt
ddpg_1.pt  ddpg_3.pt  ddpg_5.pt  ddpg_7.pt  ddpg_9.pt


In [ ]:
cccc

In [23]:
import joblib

In [24]:
SEED

0

In [26]:
joblib.dump(study, "final_ddpg_study_a_c_diff_seed_0_.pkl")

['final_ddpg_study_a_c_diff_seed_0_.pkl']

In [ ]:
# from finrl import config

In [27]:
#Get the best hyperparamters
print('Hyperparameters after tuning',study.best_params)
# print('Hyperparameters before tuning',config.DDPG_PARAMS)

Hyperparameters after tuning {'buffer_size': 1000000, 'learning_rate': 0.03991178541854849, 'batch_size': 512, 'net_arch': 'medium'}


In [28]:
study.best_trial

FrozenTrial(number=3, values=[2.91729969004257], datetime_start=datetime.datetime(2022, 7, 29, 7, 16, 0, 628849), datetime_complete=datetime.datetime(2022, 7, 29, 7, 28, 2, 299906), params={'buffer_size': 1000000, 'learning_rate': 0.03991178541854849, 'batch_size': 512, 'net_arch': 'medium'}, distributions={'buffer_size': CategoricalDistribution(choices=(10000, 100000, 1000000)), 'learning_rate': LogUniformDistribution(high=0.1, low=1e-05), 'batch_size': CategoricalDistribution(choices=(32, 64, 128, 256, 512)), 'net_arch': CategoricalDistribution(choices=('small', 'medium', 'big'))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=3, state=TrialState.COMPLETE, value=None)

In [ ]:
# from stable_baselines3 import DDPG
# tuned_model_ddpg = DDPG.load('models/ddpg_{}.pth'.format(study.best_trial.number),env=env_train)

In [20]:
# from config import *

agent = Agent(alpha= 0.03991178541854849, beta=0.03991178541854849, ckp_dir='', input_dims=state_space, tau=0.001,
              batch_size=512, layer1_size=256, layer2_size=256, max_size=1000000,
              n_actions=stock_dimension)
            
_ = agent.load_checkpoint('./models/ddpg_{}.pt'.format(3))
# _ = agent.load_checkpoint('./trained_models/daily_data_lin_tuned_seed_0/agent_ep_10.pt')

Loading checkpoint...(./trained_models/daily_data_lin_tuned_seed_0/agent_ep_10.pt)
Loaded model at episode 11 from ./trained_models/daily_data_lin_tuned_seed_0/agent_ep_10.pt


In [ ]:
_

In [ ]:
#Trading period account value with tuned model
# df_account_value_tuned, df_actions_tuned = DRLAgent.DRL_prediction(
#     model=tuned_model_ddpg, 
#     environment = e_trade_gym)

In [21]:
df_account_value_tuned, df_actions_tuned, cum_rewards_test = trade_on_test_df(df=trade, model=agent, train_df=train, env_kwargs=env_kwargs)

hit end!


In [31]:
df_account_value_tuned

,date,account_value
0,2016-01-04,1.000000e+06
1,2016-01-05,1.000269e+06
2,2016-01-06,9.984094e+05
3,2016-01-07,9.935191e+05
4,2016-01-08,9.913347e+05
...,...,...
1089,2020-05-01,1.661886e+06
1090,2020-05-04,1.666838e+06
1091,2020-05-05,1.677156e+06
1092,2020-05-06,1.664127e+06


In [32]:
def add_trade_perf_metric(df_actions, 
                          perf_stats_all,
                          trade, 
                          tp_metric):
    tpm = calc_trade_perf_metric(df_actions,trade,tp_metric)
    trp_metric = {'Value':tpm}
    df2 = pd.DataFrame(trp_metric,index=['Trade_Perf'])
    perf_stats_all = perf_stats_all.append(df2)
    return perf_stats_all

In [33]:
os.makedirs('./tuning', exist_ok=True)

In [34]:
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')
df_actions_tuned.to_csv("./"+"tuning"+"/tuned_actions_" +now+ '.csv')

In [35]:
now

'20220729-08h24'

In [36]:
#Backtesting with our pruned model
print("==============Get Backtest Results===========")
print("==============Pruned Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all_tuned = backtest_stats(account_value=df_account_value_tuned)
perf_stats_all_tuned = pd.DataFrame(perf_stats_all_tuned)
perf_stats_all_tuned.columns = ['Value']
# add trade performance metric
perf_stats_all_tuned = \
  add_trade_perf_metric(df_actions_tuned,
                        perf_stats_all_tuned,
                        trade,
                        tp_metric)
perf_stats_all_tuned.to_csv("./"+"tuning"+"/perf_stats_all_tuned_"+now+'.csv')

==============Get Backtest Results===========
==============Pruned Model===========
Annual return          0.126872
Cumulative returns     0.679584
Annual volatility      0.139565
Sharpe ratio           0.926881
Calmar ratio           0.684726
Stability              0.934761
Max drawdown          -0.185289
Omega ratio            1.195467
Sortino ratio          1.277331
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.993224
Daily value at risk   -0.017070
dtype: float64
Number of tickers: 29
Tickers: ['AAPL' 'AMGN' 'AXP' 'BA' 'CAT' 'CRM' 'CSCO' 'CVX' 'DIS' 'GS' 'HD' 'HON'
 'IBM' 'INTC' 'JNJ' 'JPM' 'KO' 'MCD' 'MMM' 'MRK' 'MSFT' 'NKE' 'PG' 'TRV'
 'UNH' 'V' 'VZ' 'WBA' 'WMT']
Calculating profit/loss for each ticker
Ratio Avg Win/Avg Loss: 2.7284904385312796


/home/neeraj/anaconda3/envs/ddpg_trading/lib/python3.8/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()
/tmp/ipykernel_5265/490943077.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  perf_stats_all = perf_stats_all.append(df2)


In [24]:
from config import BASELINE_TICKER_NAME_BACKTESTING

In [38]:
# DJI for the same time
results_df = get_comparison_df(df_account_value_tuned, BASELINE_TICKER_NAME_BACKTESTING)
train_values = np.zeros(len(results_df))
train_values[list(results_df.metric).index('Cumulative returns')] = cum_rewards_test[-1]
train_values[list(results_df.metric).index('Max drawdown')] = min(cum_rewards_test)
results_df['train_data'] = train_values

/home/neeraj/anaconda3/envs/ddpg_trading/lib/python3.8/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


Annual return          0.126872
Cumulative returns     0.679584
Annual volatility      0.139565
Sharpe ratio           0.926881
Calmar ratio           0.684726
Stability              0.934761
Max drawdown          -0.185289
Omega ratio            1.195467
Sortino ratio          1.277331
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.993224
Daily value at risk   -0.017070
dtype: float64
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (1093, 8)
Annual return          0.077076
Cumulative returns     0.379948
Annual volatility      0.200509
Sharpe ratio           0.471618
Calmar ratio           0.207830
Stability              0.796212
Max drawdown          -0.370862
Omega ratio            1.114135
Sortino ratio          0.645466
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.820444
Daily value at risk   -0.024886
dtype: float64


/home/neeraj/anaconda3/envs/ddpg_trading/lib/python3.8/site-packages/finrl/meta/preprocessor/yahoodownloader.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)
/home/neeraj/anaconda3/envs/ddpg_trading/lib/python3.8/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


In [39]:
# SAVING THIS DF_ACCOUNT_VALUE_TUNED

account_value_csv_name = f'account_value_test_tuning.csv'
actions_csv_name = f'daily_actions_test_tuning.csv'
results_table_name = f'return_comparison_test.csv'
df_account_value_tuned.to_csv(os.path.join('./tuning', account_value_csv_name))
df_actions_tuned.to_csv(os.path.join('./tuning', actions_csv_name))
results_df.to_csv(os.path.join('./tuning', results_table_name))

In [ ]:
#Now train with not tuned hyperaparameters
#Default config.ddpg_PARAMS
non_tuned_model_ddpg = agent.get_model("ddpg",model_kwargs = config.DDPG_PARAMS )
trained_ddpg = agent.train_model(model=non_tuned_model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=total_timesteps)

In [ ]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym)

In [ ]:
#Backtesting for not tuned hyperparamters
print("==============Get Backtest Results===========")
print("============Default Hyperparameters==========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.columns = ['Value']
# add trade performance metric
perf_stats_all = add_trade_perf_metric(df_actions,
                        perf_stats_all,
                        trade,
                        tp_metric)
perf_stats_all.to_csv("./"+"tuning"+"/perf_stats_all_untuned_"+now+'.csv')

In [ ]:
study = joblib.load("final_ddpg_study__.pkl")

In [ ]:
#Certainly you can afford more number of trials for further optimization
from optuna.visualization import plot_optimization_history
fig = plot_optimization_history(study)
#"./"+config.RESULTS_DIR+
fig.write_image("./"+"tuning"+"/opt_hist.png")
fig.show()

In [ ]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [ ]:
#Hyperparamters importance

try:
    fig = plot_param_importances(study)
    fig.write_image("./"+config.RESULTS_DIR+"/params_importances.png")
    fig.show()
except:
    print('Cannot calculate hyperparameter importances: no variation')

In [ ]:
# fig = plot_edf(study)
# fig.write_image("./"+"tuning"+"/emp_dist_func.png")
# fig.show()